# 03 Support Vector Machines

In [ ]:
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import classification_report, accuracy_score

# Classical Implementation

Text

In [1]:
params = {
    'kernel': 'rbf', # 'rbf','linear', 'poly', 'sigmoid'
    'C': 1,
    'dir': '',
}

In [ ]:
os.makedirs(f"{os.getcwd()}/data", exist_ok=True)

if not os.path.exists(f"{os.getcwd()}/data/iris.csv"):
  print("Downloading iris dataset...")
  iris = fetch_ucirepo(id=53)

  X = iris.data.features
  Y = iris.data.targets['class']
  print("Saving iris dataset to ~/data/iris.csv...")
  pd.concat([X, Y], axis=1).to_csv(f'{os.getcwd()}/data/iris.csv', index=False)
else:
  print("Loading iris dataset from ~/data/iris.csv...")
  iris_df = pd.read_csv(f'{os.getcwd()}/data/iris.csv')
  X = iris_df.drop('class', axis=1)
  Y = iris_df['class']

print("\nx type:", type(X))
print("y type:", type(Y))
print("\nFirst few rows of x:\n", X.head())
print("\nFirst few rows of y:\n", Y.head())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm = SVC(kernel=params['kernel'], C=params['C'], random_state=42)

cv_scores = cross_val_score(svm, X_train_scaled, y_train, cv=5)

print("Cross-validation scores for each fold:", cv_scores)
print("\nAverage cross-validation score:", cv_scores.mean())
print("Standard deviation of cross-validation scores:", cv_scores.std())

start_time = time.time()
svm.fit(X_train_scaled, y_train)
end_time = time.time()

training_time = end_time - start_time
print(f"\nModel training time: {training_time:.4f} seconds")

y_pred = svm.predict(X_test_scaled)

print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nAccuracy on test set:", accuracy_score(y_test, y_pred))

In [ ]:
del params, X_train, X_test, y_train, y_test, scaler, X_train_scaled, X_test_scaled, svm, cv_scores, start_time, end_time, training_time, y_pred

In [ ]:
params = {
  'kernel': ['rbf','linear', 'poly', 'sigmoid'], # ['rbf','linear', 'poly', 'sigmoid']
  'C': [1],
}

In [ ]:
X_split = X.iloc[:, :2]

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(Y)

X_train, X_test, y_train, y_test = train_test_split(X_split, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
def run_svm_experiment(kernel, c, X_train_scaled, X_test_scaled, y_train, y_test):
    title_map = {
      'rbf': 'SVM with RBF Kernel',
      'linear': 'SVM with Linear Kernel',
      'poly': 'SVM with Polynomial Kernel',
      'sigmoid': 'SVM with Sigmoid Kernel'
    }
    svm = SVC(kernel=f'{kernel}', C=c, random_state=42)
    start_time = time.time()
    svm.fit(X_train_scaled, y_train)
    end_time = time.time()
    training_time = end_time - start_time
    y_pred = svm.predict(X_test_scaled)

    plt.figure(figsize=(10, 6))

    DecisionBoundaryDisplay.from_estimator(
      svm,
      X_train_scaled,
      response_method="predict",
      cmap=plt.cm.coolwarm,
      alpha=0.75
    )

    scatter_train = plt.scatter(X_train_scaled[:, 0], X_train_scaled[:, 1], c=y_train, marker='o', s=50, cmap=plt.cm.coolwarm)
    scatter_test = plt.scatter(X_test_scaled[:, 0], X_test_scaled[:, 1], c=y_test, marker='x', s=100, cmap=plt.cm.coolwarm)

    plt.xlabel('Sepal length (cm)')
    plt.ylabel('Sepal width (cm)')
    plt.title(f'{title_map[f"{kernel}"]}')

    handles, labels = scatter_train.legend_elements()
    handles_class, labels_class = scatter_train.legend_elements()
    train_handle = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=10, label='Train')
    test_handle = plt.Line2D([0], [0], marker='x', color='w', markerfacecolor='black', markersize=10, label='Test')
    handles = handles_class + [train_handle, test_handle]
    labels = labels_class + ['Train', 'Test']
    plt.legend(handles=handles, labels=labels, loc="upper left")

    print(f"\nModel training time: {training_time:.4f} seconds")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print("\nAccuracy on test set:", accuracy_score(y_test, y_pred))

    os.makedirs(f"{os.getcwd()}/plots", exist_ok=True)
    plot_filename = os.path.join(f"{os.getcwd()}/plots", f"svm_{kernel}_c{c}.png")
    print(f"\nSaving plot to {plot_filename}...")
    plt.savefig(plot_filename)

    plt.show()

In [ ]:
for kernel in params['kernel']:
  for c in params['C']:
    if kernel not in ['rbf','linear', 'poly', 'sigmoid']:
      print(f"Skipping invalid kernel: {kernel}")
      continue
    if not isinstance(c, (float, int)):
      print(f"Skipping invalid C: {c}")
      continue
    run_svm_experiment(kernel=kernel,
                       c = c,
                       X_train_scaled=X_train_scaled,
                       X_test_scaled=X_test_scaled,
                       y_train=y_train,
                       y_test=y_test)

# Quantum Implementation

Text